In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup

In [2]:
#scrape understat player
base_url = 'https://understat.com/player/'
player_id = '5656'
url = base_url + player_id

In [3]:
#Use requests to get the webpage and BeautifulSoup to parse the page
#Beautiful Soup is a Python library for getting data out of HTML, XML, and other markup languages
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [4]:
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var groupsData 		= JSON.parse('\x7B\x22season\x22\x3A\x5B\x7B\x22position\x22\x3A\x22Sub\x22,\x22games\x22\x3A\x2213\x22,\x22goals\x22\x3A\x223\x22,\x22shots\x22\x3A\x2216\x22,\x22time\x22\x3A\x22561\x22,\x22xG\x22\x3A\x221.880533360876143\x22,\x22assists\x22\x3A\x220\x22,\x22xA\x22\x3A\x220.13034621253609657\x22,\x22key_passes\x22\x3A\x222\x22,\x22season\x22\x3A\x222020\x22,\x22team\x22\x3A\x22Arsenal\x22,\x22yellow\x22\x3A\x221\x22,\x22red\x22\x3A\x221\x22,\x22npg\x22\x3A\x222\x22,\x22npxG\x22\x3A\x221.1193645410239697\x22,\x22xGChain\x22\x3A\x221.6147465873509645\x22,\x22xGBuildup\x22\x3A\x220.6273571960628033\x22\x7D,\x7B\x22position\x22\x3A\x22Sub\x22,\x22games\x22\x3A\x2231\x22,\x22goals\x22\x3A\x225\x22,\x22shots\x22\x3A\x2250\x22,\x22time\x22\x3A\x222021\x22,\x22xG\x22\x3A\x225.069305270910263\x22,\x22assists\x22\x3A\x225\x22

In [5]:
string_with_json_obj = ''

# Find shotsdata only
for el in scripts:
    if 'shotsData' in str(el):
        string_with_json_obj = str(el).strip()

#print(string_with_json_obj)
# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [6]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
data

[{'id': '103860',
  'minute': '62',
  'result': 'MissedShots',
  'X': '0.87',
  'Y': '0.32',
  'xG': '0.0549338161945343',
  'player': 'Nicolas Pepe',
  'h_a': 'a',
  'player_id': '5656',
  'situation': 'OpenPlay',
  'season': '2016',
  'shotType': 'LeftFoot',
  'match_id': '2272',
  'h_team': 'Metz',
  'a_team': 'Angers',
  'h_goals': '2',
  'a_goals': '0',
  'date': '2016-08-27 22:00:00',
  'player_assisted': 'Yoann Andreu',
  'lastAction': 'Pass'},
 {'id': '103864',
  'minute': '85',
  'result': 'BlockedShot',
  'X': '0.915',
  'Y': '0.39599998474121095',
  'xG': '0.07553026080131531',
  'player': 'Nicolas Pepe',
  'h_a': 'a',
  'player_id': '5656',
  'situation': 'SetPiece',
  'season': '2016',
  'shotType': 'RightFoot',
  'match_id': '2272',
  'h_team': 'Metz',
  'a_team': 'Angers',
  'h_goals': '2',
  'a_goals': '0',
  'date': '2016-08-27 22:00:00',
  'player_assisted': None,
  'lastAction': 'None'},
 {'id': '104598',
  'minute': '39',
  'result': 'BlockedShot',
  'X': '0.8069999

In [7]:
#make the dataframe column
season = []
minute = []
result = []
x = []
y = []
xG = []
situation = []
shotType = []
col_names = ['Season', 'Minute', 'Result', 'x', 'y', 'xG', 'Situation', 'ShotType']
for index in range(len(data)):
    for key in data[index]:
        if key == 'season':
            season.append(data[index][key])
        if key == 'minute':
            minute.append(data[index][key])
        if key == 'result':
            result.append(data[index][key])
        if key == 'X':
            x.append(data[index][key])
        if key == 'Y':
            y.append(data[index][key])
        if key == 'xG':
            xG.append(data[index][key])
        if key == 'situation':
            situation.append(data[index][key])
        if key == 'shotType':
            shotType.append(data[index][key])

In [8]:
df = pd.DataFrame([season,minute,result,x,y,xG,situation,shotType],index=col_names)
df = df.T

In [9]:
df.head()

,Season,Minute,Result,x,y,xG,Situation,ShotType
0,2016,62,MissedShots,0.87,0.32,0.0549338161945343,OpenPlay,LeftFoot
1,2016,85,BlockedShot,0.915,0.39599998474121095,0.07553026080131531,SetPiece,RightFoot
2,2016,39,BlockedShot,0.8069999694824219,0.5209999847412109,0.05410444363951683,OpenPlay,LeftFoot
3,2016,51,BlockedShot,0.8859999847412109,0.31,0.04117557406425476,OpenPlay,LeftFoot
4,2016,77,MissedShots,0.7709999847412109,0.405,0.02198970504105091,OpenPlay,LeftFoot


In [11]:
#exporting to CSV file
df.to_csv('pepe_shotdata.csv')